## Basic Setup

In [ ]:
# source for utility functions: get_device, get_cifar10_loaders, get_resnet50_for_cifar10
#                               train, evaluate, estimate_latency, get_size
%run utils.ipynb



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
from copy import deepcopy

# get device
device = get_device()

# get the CIFAR-10 data loaders
train_loader, val_loader, test_loader = get_cifar10_loaders()

# Get architecture for CIFAR-10 training
model = get_resnet50_for_cifar10(device)

Full train set size: 50000
Train ratio: 0.9
Train samples: 45000
Validation samples: 5000
Test samples: 10000
Number of training batches: 352
Number of validation batches: 40
Number of test batches: 79


## Train and Evaluate full model

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

train(
    model,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    device,
    epochs=50,
    scheduler=scheduler,
    grad_clip=1.0,
    save_path="full_model_resnet50_best_model.pt",
    early_stopping_patience=5,
    resume=True,
)

# Save the fine-tuned original model
original_model = deepcopy(model)

🔁 Resumed training from epoch 4


Epoch   5 | Train Loss: 0.4172 | Acc: 0.8527
          | Val   Loss: 0.6039 | Acc: 0.8130
          | ✅ New best model saved to 'full_model_resnet50_best_model.pt'


Epoch   6 | Train Loss: 0.3214 | Acc: 0.8864
          | Val   Loss: 0.6077 | Acc: 0.8152
          | No improvement for 1 epoch(s)


Epoch   7 | Train Loss: 0.2445 | Acc: 0.9149
          | Val   Loss: 0.6085 | Acc: 0.8206
          | No improvement for 2 epoch(s)


Epoch   8 | Train Loss: 0.1817 | Acc: 0.9368
          | Val   Loss: 0.6411 | Acc: 0.8256
          | No improvement for 3 epoch(s)


Epoch   9 | Train Loss: 0.1383 | Acc: 0.9521
          | Val   Loss: 0.6303 | Acc: 0.8366
          | No improvement for 4 epoch(s)


Epoch  10 | Train Loss: 0.0373 | Acc: 0.9880
          | Val   Loss: 0.6574 | Acc: 0.8560
          | No improvement for 5 epoch(s)
🛑 Early stopping triggered after 5 epochs without improvement.
Training complete.


In [26]:
# Function to compress layers
def compress_layer(layer, epsilon=0.10):
    """
    Compresses a layer using SVD if the compression is beneficial.
    Args:
        layer (nn.Module): The layer to compress.
        epsilon (float): The energy threshold for compression.
    Returns:
        nn.Module: The compressed layer or the original layer if compression is not beneficial.
    """

    # handle Linear layers
    if isinstance(layer, nn.Linear):
        # get linear layer weight matrix
        W = layer.weight.data.cpu()
        
        # run SVD on flat weight matrix
        U, S, Vh = torch.linalg.svd(W, full_matrices=False)

        # find rank that capture the asked energy (1-epsilon)
        energy = torch.cumsum(S**2, dim=0) / torch.sum(S**2)
        rank = torch.searchsorted(energy, 1 - epsilon).item() + 1

        # check that factorization actually reduces number of parameters
        old_size = W.numel()
        new_size = rank * (W.shape[0] + W.shape[1])
        if new_size < old_size:
            # define low rank factorization from SVD and rank
            U_r = U[:, :rank] @ torch.diag(S[:rank])
            V_r = Vh[:rank, :]

            # define two linear layers to replace the original linear layer
            compressed_layer = nn.Sequential(
                nn.Linear(W.shape[1], rank, bias=False),
                nn.Linear(rank, W.shape[0], bias=True)
            )
            compressed_layer[0].weight.data = V_r.to(device)
            compressed_layer[1].weight.data = U_r.to(device)
            compressed_layer[1].bias.data = layer.bias.data.to(device)
            return compressed_layer, old_size, new_size
        
    # handle Conv2d layers
    elif isinstance(layer, nn.Conv2d):
        # get convolution weight 4d matrix, shape: [out_channels, in_channels, kH, kW]
        W = layer.weight.data.cpu()  
        OC, IC, kH, kW = W.shape

        # reshape to 2d matrix, with shape: [OC, IC*kH*kW]
        W_flat = W.view(OC, -1)

        # run SVD on flat weight matrix        
        U, S, Vh = torch.linalg.svd(W_flat, full_matrices=False)

        # find rank that capture the asked energy (1-epsilon)
        energy = torch.cumsum(S**2, dim=0) / torch.sum(S**2)
        rank = torch.searchsorted(energy, 1 - epsilon).item() + 1

        # check that factorization actually reduces number of parameters
        old_size = W.numel()
        new_size = rank * (IC * kH * kW + OC)
        if new_size < old_size:
            # define low rank factorization from SVD and rank
            U_r = U[:, :rank] @ torch.diag(S[:rank])
            V_r = Vh[:rank, :]

            # define two convolutional layers to replace the original convolutional layer
            conv1 = nn.Conv2d(
                in_channels=IC,
                out_channels=rank,
                kernel_size=1,
                stride=1,
                padding=0,
                bias=False
            )
            conv2 = nn.Conv2d(
                in_channels=rank,
                out_channels=OC,
                kernel_size=(kH, kW),
                stride=layer.stride,
                padding=layer.padding,
                bias=(layer.bias is not None)
            )
            conv1.weight.data = V_r.view(rank, IC, kH, kW).to(device)
            conv2.weight.data = U_r.view(OC, rank, 1, 1).to(device)
            if layer.bias is not None:
                conv2.bias.data = layer.bias.data.to(device)
            return nn.Sequential(conv1, conv2), old_size, new_size

    return layer, 0, 0  # return the original layer if compression is not beneficial

In [27]:
def compress_model(model, epsilon=0.50):
    """
    Compresses the given model by applying SVD-based compression to Linear and Conv2d layers.
    
    Args:
        model (nn.Module): The model to compress.
        epsilon (float): The energy threshold for compression.
    
    Returns:
        nn.Module: The compressed model.
    """
   
    compressed_model = deepcopy(model)  # Create a copy of the input model

    total_old_size = 0
    total_new_size = 0

    for name, module in compressed_model.named_modules():
        if isinstance(module, (nn.Linear, nn.Conv2d)):
            if '.' in name:  # Check if the module has a parent
                parent, attr = name.rsplit('.', 1)
                parent_module = compressed_model
                for part in parent.split('.'):
                    parent_module = getattr(parent_module, part)
            else:  # Handle top-level modules
                parent_module = compressed_model
                attr = name
            new_layer, old_size, new_size = compress_layer(module, epsilon)
            total_old_size += old_size
            total_new_size += new_size
            setattr(parent_module, attr, new_layer)
    
    return compressed_model, total_old_size, total_new_size

In [29]:
# Evaluate and print metrics for the original model
acc_orig = evaluate(original_model, test_loader, device)
example_input = torch.rand(128, 3, 32, 32).to(device)
orig_latency_mu, orig_latency_std = estimate_latency(original_model, example_input)
size_orig = get_size(original_model)
print(f"Original -> acc: {100*acc_orig:.2f}%, latency: {orig_latency_mu:.2f} ± {orig_latency_std:.2f} ms, size: {size_orig:.2f}MB")

# Iterate over epsilon values
for epsilon in [round(x * 0.1, 2) for x in range(1, 10)]:
    print(f"\nCompressing model with epsilon = {epsilon}...")
    
    # Compress the model
    compressed_model, total_old_size, total_new_size = compress_model(original_model, epsilon=epsilon)
    
    # Evaluate compressed model before fine-tuning
    acc_comp = evaluate(compressed_model, test_loader, device)
    print(f"Old size: {total_old_size}, New size: {total_new_size}, Parameter count reduction: {total_old_size-total_new_size}")
    print(f"Compressed -> acc before tuning: {100*acc_comp:.2f}%")
    
    # Fine-tune the compressed model
    optimizer = torch.optim.Adam(compressed_model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)
    
    train(
        compressed_model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        device,
        epochs=50,
        scheduler=scheduler,
        grad_clip=1.0,
        save_path=f"compressed_model_epsilon_{epsilon}_best_model.pt",
        early_stopping_patience=3,
        resume=False,
    )
    
    # Evaluate compressed model after fine-tuning
    acc_tuned_comp = evaluate(compressed_model, test_loader, device)
    comp_latency_mu, comp_latency_std = estimate_latency(compressed_model, example_input)
    size_comp = get_size(compressed_model)
    
    # Print metrics for the fine-tuned compressed model
    print(f"Compressed -> acc after tuning: {100*acc_tuned_comp:.2f}%, latency: {comp_latency_mu:.2f} ± {comp_latency_std:.2f} ms, size: {size_comp:.2f}MB")

Measuring latency: 100%|██████████| 50/50 [00:03<00:00, 12.74it/s]


Original -> acc: 84.63%, latency: 72.00 ± 0.06 ms, size: 94.38MB

Compressing model with epsilon = 0.1...


Evaluating: 100%|██████████| 79/79 [00:07<00:00, 10.14batch/s]


Old size: 23463616, New size: 17327424, Parameter count reduction: 6136192
Compressed -> acc before tuning: 34.17%


Epoch   1 | Train Loss: 0.4461 | Acc: 0.8448
          | Val   Loss: 0.8771 | Acc: 0.7516
          | ✅ New best model saved to 'compressed_model_epsilon_0.1_best_model.pt'


...

Epoch   9 | Train Loss: 1.5557 | Acc: 0.3587
          | Val   Loss: 1.5770 | Acc: 0.3556
          | No improvement for 1 epoch(s)


Epoch  10 | Train Loss: 1.5352 | Acc: 0.3704
          | Val   Loss: 1.5570 | Acc: 0.3484
          | No improvement for 2 epoch(s)


Epoch  11 | Train Loss: 1.5219 | Acc: 0.3750
          | Val   Loss: 1.6016 | Acc: 0.3384
          | No improvement for 3 epoch(s)
🛑 Early stopping triggered after 3 epochs without improvement.
Training complete.


Measuring latency: 100%|██████████| 50/50 [00:03<00:00, 14.22it/s]


Compressed -> acc after tuning: 35.11%, latency: 66.13 ± 0.06 ms, size: 1.37MB
